In [7]:
from datautil.common import path_to_save_base
from datautil.gamelog import result_parse

In [14]:
for i in range(10):
    print(i)
    i+=2

0
1
2
3
4
5
6
7
8
9


In [8]:
import os
save = 'test_1000'
fn = path_to_save_base(save) + "/news/txt/leagues/log_1.txt"

logs = open(fn).readlines()

In [71]:
import re
cursor = 0
inning = (0, 0)
pitcher_id = None 
batter_id = None

watch = (4,0)

stack = []
last_result = None
while cursor < len(logs):
    ln = logs[cursor]

    res_inn = re.search("(Top|Bottom) of the ([0-9]+)(st|nd|rd|th) -", ln)
    if res_inn != None:
        inning = (int(res_inn.group(2)), 0 if res_inn.group(1)=='Top' else 1)
        if inning >= watch:
            print ("-"*32)
            print ("Inning change :", inning)
        cursor += 1
        continue

    if inning < watch:
        cursor +=1
        continue
    
    res_p = re.search("Pitching: .HP <a href=\"../players/player_(\d+).html", ln)
    if res_p != None:
        pitcher_id = int(res_p.group(1))
        print ("Pitcher", pitcher_id, "for home team" if inning[1]=="Top" else "for road team")
        cursor += 1
        continue
    
    res_b = re.search("Batting: .HB <a href=\"../players/player_(\d+).html", ln)
    if res_b != None:
        if len(stack) > 0 and last_result == None:
            print("".join(["*** " + a for a in stack]))

        batter_id = int(res_b.group(1))
        print ("- Batter", batter_id)
        cursor += 1
        stack = []
        last_result = None
        continue

    # groud out
    res = re.search("Ground out ([^ ]*) \((.*), (.*), EV (.*) MPH\)", ln)
    if res != None:
        last_result = "--- GO > def:{} type:{} loc:{} spd:{}".format(*res.groups())
        print(last_result)

    # fly out --> could be sacrifice fo
    res = re.search("Fly out, ([^ ]*) *\((.*), (.*), EV (.*) MPH\)", ln)
    if res != None:
        last_result = "--- FO > def:{} type:{} loc:{} spd:{}".format(*res.groups())
        print(last_result)
        if "Runner from 3rd tags up, SCORES" in logs[cursor + 1]:
            print ("--- Rec SF")
    
    # GIDP 
    res = re.search("Grounds into double play, ([^ ]*) *\((.*), (.*), EV (.*) MPH\)", ln)
    if res != None:
        last_result = "--- GO > def:{} type:{} loc:{} spd:{}".format(*res.groups())
        print(last_result)
        print("--- GIDP")

    # FC
    res = re.search("Fielders Choice at ..., (.*) \((.*), (.*), EV (.*) MPH\)", ln)
    if res != None:
        last_result = "--- FC > def:{} type:{} loc:{} spd:{}".format(*res.groups())
        print(last_result)

    # hit single/double/triple
    res = re.search("<b>(SINGLE|DOUBLE|TRIPLE)</b>  \((.*), (.*), EV (.*) MPH\)", ln)
    if res != None:
        last_result = "--- HIT({}) > type:{} loc:{} spd:{}".format(*res.groups())
        print(last_result)

    # hit HR
    res = re.search("<b>([A-Z]*) HOME RUN</b>  \((.*), (.*), EV (.*) MPH\), Distance : (.*) ft", ln)
    if res != None:
        last_result = "--- HR > score:{} type:{} loc:{} spd:{} dist:{}".format(*res.groups())
        print(last_result)

    # strike out
    res = re.search("Strikes out (looking|swinging)", ln)
    if res != None:
        last_result = "--- SO > {}".format(*res.groups())
        print(last_result)

    # reached on error
    res = re.search("Reached on error, E(.) \((.*), (.*), EV (.*) MPH\)", ln)
    if res != None:
        last_result = "--- ERR > def:{} type:{} loc:{} spd:{}".format(*res.groups())
        print(last_result)

    res = re.search("Reached via error on a dropped throw from (.*), E(.) \((.*), (.*), EV (.*) MPH\)", ln)
    if res != None:
        last_result = "--- ERR (drop throw) > def:{} err:{} type:{} loc:{} spd:{}".format(*res.groups())
        print(last_result)

    if "Hit by Pitch" in ln:
        last_result = "--- HBP"
        print(last_result)

    stack.append(ln)

    cursor += 1


--------------------------------
Inning change : (4, 0)
- Batter 218
--- GO > def:6-3 type:Groundball loc:6 spd:85.6
- Batter 213
--- SO > swinging
- Batter 212
--- SO > looking
--------------------------------
Inning change : (4, 1)
- Batter 28
--- HR > score:SOLO type:Flyball loc:89XD spd:106.3 dist:408
- Batter 33
--- GO > def:U3 type:Groundball loc:3S spd:73.8
- Batter 1375
--- FO > def:F8 type:Flyball loc:8LM spd:76.9
- Batter 29
--- FO > def:F8 type:Flyball loc:8LD spd:109.4
--------------------------------
Inning change : (5, 0)
- Batter 219
--- FO > def:F8 type:Flyball loc:8RM spd:83.1
- Batter 203
--- HIT(1) > type:Flyball loc:34D spd:87.5
- Batter 214
--- HIT(1) > type:Groundball loc:34D spd:106.9
- Batter 207
--- HIT(1) > type:Groundball loc:4M spd:107.5
- Batter 190
--- HIT(1) > type:Line Drive loc:34D spd:107.5
- Batter 211
--- FO > def:F8 type:Line Drive loc:8LD spd:86.3
--- Rec SF
- Batter 218
--- FC > def:6-4 type:Groundball loc:6 spd:89.4
------------------------------